In [1]:
import sys
sys.path.append('..')

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch

from evodiff.pretrained import OA_DM_38M, OA_DM_640M
from evodiff.generate import generate_oaardm
from evodiff.conditional_generation import inpaint_simple 

torch.hub.set_dir('/data/users/kgeorge/workspace/evodiff')

/nethome/kgeorge/miniconda3/envs/workspace-esm/lib/python3.10/site-packages/torch_geometric/typing.py:54: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.29' not found (required by /nethome/kgeorge/miniconda3/envs/workspace-esm/lib/python3.10/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/nethome/kgeorge/miniconda3/envs/workspace-esm/lib/python3.10/site-packages/torch_geometric/typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.29' not found (required by /nethome/kgeorge/miniconda3/envs/workspace-esm/lib/python3.10/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [3]:
from DomainPrediction import BaseProtein
from DomainPrediction.eval import metrics
from DomainPrediction.utils import helper
from DomainPrediction.utils.constants import *

In [4]:
protein = BaseProtein(file='../../Data/6mfw_conformations/hm_6mfz_ATC.pdb')

In [5]:
protein.get_residues(T_6mfw) ## T domain

'QQLAQVWSHVLGIPQMGIDDHFLERGGDSIKVMQLIHQLKNIGLSLRYDQLFTHPTIRQLKRLL'

In [6]:
T_6mfw[0], T_6mfw[-1]+1

(492, 556)

In [7]:
# checkpoint = OA_DM_38M()
checkpoint = OA_DM_640M()
model, collater, tokenizer, scheme = checkpoint

In [8]:
model = model.cuda()

In [9]:
sequence = protein.sequence
start_idx, end_idx = T_6mfw[0], T_6mfw[-1]+1

fasta_file = '../../Data/evodiff_experiments/6mfw_exp/6mfw_evodiff_1000.fasta'
for i in range(1000):
    seq_dict = {}
    sample, entire_sequence, generated_idr = inpaint_simple(model, sequence, start_idx, end_idx, tokenizer=tokenizer, device='cuda')
    id = f'6mfw_ATC_evodiff_gen_{i}'
    seq_dict[id] = entire_sequence

    helper.create_fasta(seq_dict, fasta_file, append=True)


100%|██████████| 64/64 [00:39<00:00,  1.61it/s]


In [ ]:
# fasta_file = '../../Data/evodiff_experiments/6mfw_exp/6mfw_evodiff_1000.fasta'
# helper.create_fasta(seq_dict, fasta_file)

In [10]:
records = helper.read_fasta(fasta_file)
seq_dict_ = {}
for rec in records:
    seq_dict_[rec.id] = ''.join([str(rec.seq)[i] for i in T_6mfw])

file = '../../Data/evodiff_experiments/6mfw_exp/6mfw_evodiff_1000.T.fasta'
helper.create_fasta(seq_dict_, file)